In [7]:
import pandas as pd
from docx import Document
import base64
from io import BytesIO
from PIL import Image


In [8]:
df = pd.read_csv('recieps.csv')

In [9]:
SIGN_COL = "חתימה"
AMOUNT_COL = "סכום"
NAME_COL = "שם"
ID_COL = "תעודת זהות"
DATA_COL = "responseDate"

In [10]:
TITLE = "חתימת נבדק"

In [11]:
def create_doc(row):
    doc = Document()
    # Set RTL properties for the entire document
    doc.styles['Normal'].font.rtl = True
    doc.add_heading(TITLE, 0)
    doc.add_paragraph(f"שם: {row[NAME_COL]}")
    doc.add_paragraph(f"תעודת זהות: {row[ID_COL]}")
    doc.add_paragraph(f"סכום: {row[AMOUNT_COL]}")
    doc.add_paragraph(f"תאריך: {row[DATA_COL]}")

    # Convert base64 signature to image bytes
    try:
        base64_signature = row[SIGN_COL]
        base64_signature = base64_signature.replace("data:image/png;base64,", "")
        signature_bytes = base64.b64decode(base64_signature)
        img = Image.open(BytesIO(signature_bytes))
        
        # Save the image to a temporary file
        temp_signature_path = "temp_signature.png"
        img.save(temp_signature_path)

        # Add the image to the document
        doc.add_picture(temp_signature_path)

        # Clean up: remove the temporary signature image file
        import os
        os.remove(temp_signature_path)
    except Exception as e:
        print(f"Failed to add signature for {row[NAME_COL]}")
        print(e)
        doc.add_paragraph(f"חתימה: {row[SIGN_COL]}")
    
    doc.save(f"result/{row[NAME_COL]}.docx")


In [12]:
for index, row in df.iterrows():
    create_doc(row)